# 1 - Explique o passo a passo do bagging

1. Pegar uma amostra de dados, seja em CSV ou em banco de dados.
2. A partir dessa amostra, gerar diversas outras amostras **com reposição**.
3. Treinar um modelo para cada uma dessas amostras geradas.
4. Testar todos os modelos com um registro separado para o teste, e avaliar as previsões.
- caso seja um problema de classificação, adotar como certo o que a maoiria dos modelos preveram.
- caso seja um problema de regressão, adotar como certo a média das previsões de todos os modelos.

# 2 - Explique com suas palavras o bagging 

O bagging consiste em criar múltiplas amostras com reposição a partir de uma amostra inicial, e para cada uma dessas amostras treinar um modelo (seja de classificação ou de regressão).

Com os modelos treinados, pegar um caso da base de testes e realizar uma previsão com todos os modelos treinados. Caso seja um problema de classificação, ver o que a maioria dos modelos preveu, caso seja um problema de regressão, tirar uma média da previsão de todos os modelos.  

# 3 - (Opcional) Implementar em python o código do Bagging

- Bootstrap
- Modelagem
- Agregação

In [ ]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

**Leitura do CSV de ataques cardíacos**

In [9]:
df = pd.read_csv('heart.csv')

print(df.shape)

df.head()

(303, 14)


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


**Divisão entre treino e teste**

In [3]:
df_treino, df_teste = train_test_split(df, test_size=0.10)

df_treino.shape

(272, 14)

### Etapa do bootstrap, gero 100 amostras com reposição

Gero 100 amostras com reposição

In [10]:
lista_df = []

for i in range (0, 100):
    
    df_aux = df_treino.sample(n = df_treino.shape[0], replace=True)
    
    lista_df.append(df_aux)
    
len(lista_df)

100

### Etapa da modelagem, gero um modelo pra cada uma das 100 amostras

In [11]:
lista_modelos = []

for data_frame in lista_df:
    
    clf = DecisionTreeClassifier(max_depth=10, min_samples_leaf=5)
    
    clf = clf.fit(data_frame.drop('output', axis=1), data_frame['output'])
    
    lista_modelos.append(clf)
    
len(lista_modelos)

100

### Etapa da agregação, vejo o que a maioria dos modelos retorna

Pego uma linha qualquer da base de teste

In [12]:
reg_teste = df_teste.sample(n=1)
reg_teste

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
219,48,1,0,130,256,1,0,150,1,0.0,2,2,3,0


Faço uma previsão com todos os 100 modelos criados e vejo o que a maioria retorna

In [13]:
previsoes = []

for modelo in lista_modelos:

    pred = lista_modelos[0].predict(reg_teste.drop('output', axis=1))
    
    previsoes.append(pred[0])
    
len(previsoes)

100

**Avalio qual foi a maior parte das previsões**

Neste caso, todos os 100 modelos preveram corretamente a variável resposta

In [14]:
pd.Series(previsoes).value_counts()

0    100
dtype: int64